In [12]:
!pip install -r requirements.txt

import zeep
from zeep import Client
import ipywidgets as widgets
from IPython.display import display
import webbrowser

     |████████████████████████████████| 1.6 MB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 347 kB 47.5 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 8.7 MB/s  eta 0:00:01
ERROR: nbconvert 7.4.0 has requirement jinja2>=3.0, but you'll have jinja2 2.11.2 which is incompatible.
ERROR: jupyterlab 3.2.4 has requirement nbclassic~=0.2, but you'll have nbclassic 1.0.0 which is incompatible.
ERROR: jupyterlab-server 2.23.0 has requirement jinja2>=3.0.3, but you'll have jinja2 2.11.2 which is incompatible.
ERROR: voila 0.2.10 has requirement jupyter-client<7,>=6.1.3, but you'll have jupyter-client 7.4.9 which is incompatible.
ERROR: voila 0.2.10 has requirement nbconvert<7,>=6.0.0, but you'll have nbconvert 7.4.0 which is incompatible.
  Attempting uninstall: jupyter-server
    Found existing installation: jupyter-server 2.5.0
    Uninstalling jupyter-server-2.5.0:
      Successfully uninstalled jupyter-server-2.5.0
  Attempting uninstall: nbclient
    Found ex

In [2]:
temperature_input = widgets.FloatText(description="Temperature:")
from_unit_dropdown = widgets.Dropdown(options=["degreeFahrenheit", "degreeCelsius"], description="From:")
to_unit_dropdown = widgets.Dropdown(options=["degreeCelsius", "degreeFahrenheit"], description="To:")
convert_button = widgets.Button(description="Convert")

output_text = widgets.Text(description="Converted Temperature:")
output_number = widgets.Text(description="Number Converted to Words:")
error_text = widgets.Text(description="Error:")

In [13]:
def convert_button_clicked(b):
    try:
        # Get input values
        temp = temperature_input.value
        from_unit = from_unit_dropdown.value
        to_unit = to_unit_dropdown.value
        
        # Call function to convert temperature
        converted_temp = convert_temperature(temp, from_unit, to_unit)

        # Convert the converted_temp to a float for comparison
        converted_temp_float = float(converted_temp)

        if converted_temp_float < 0:
            # Set the converted_temp to its absolute value
            converted_temp = abs(converted_temp_float)
        else:
            error_text.value = ""  # Clear any previous error message

        # Display the original converted temperature value
        output_text.value = str(converted_temp_float)
        
        # Call function to convert number to words
        number_converted = convert_number_to_words(converted_temp)
        
        # Check if the converted temperature has fractional parts other than .00
        if converted_temp_float % 1 != 0 and str(converted_temp_float % 1) != ".00":
            error_text.value = "Error: SOAP cannot convert fractional values. Temperature rounded down."
        
        if converted_temp_float < 0:
            # Display negative value with "Negative" prefix
            output_number.value = "Negative " + number_converted
        else:
            output_number.value = number_converted
                
    except Exception as e:
        error_text.value = str(e)


In [14]:
def open_web_app():
    webbrowser.open_new_tab("http://localhost:8890/notebooks/temp_converter.ipynb")

In [15]:
# WSDL URLs for SOAP services
tempconvert_wsdl = "https://www.w3schools.com/xml/tempconvert.asmx?WSDL"
numberconversion_wsdl = "https://www.dataaccess.com/webservicesserver/numberconversion.wso?WSDL"

# Create clients for both SOAP services
tempconvert_client = Client(tempconvert_wsdl)
numberconversion_client = Client(numberconversion_wsdl)


Forcing soap:address location to HTTPS
Forcing soap:address location to HTTPS
Forcing http:address location to HTTPS


In [16]:
# Function to convert temperature using SOAP service
def convert_temperature(temp, from_unit, to_unit):
    # Call the appropriate operation based on the units
    if from_unit.lower() == 'degreecelsius' and to_unit.lower() == 'degreefahrenheit':
        result = tempconvert_client.service.CelsiusToFahrenheit(temp)
    elif from_unit.lower() == 'degreefahrenheit' and to_unit.lower() == 'degreecelsius':
        result = tempconvert_client.service.FahrenheitToCelsius(temp)
    else:
        raise ValueError(f"Invalid temperature conversion: {from_unit} to {to_unit}. Please try again.")

    return result

In [17]:
# Function to convert number into words using SOAP service
def convert_number_to_words(number):
    result = numberconversion_client.service.NumberToWords(number)
    return result

In [18]:
open_app_button = widgets.Button(description="Open Web App in Separate Window")
open_app_button.on_click(lambda b: open_web_app())

# Register the event handler for the convert button
convert_button.on_click(convert_button_clicked)

# Display the widgets
display(temperature_input, from_unit_dropdown, to_unit_dropdown, convert_button, output_text, output_number, error_text, open_app_button)

FloatText(value=0.0, description='Temperature:')

Dropdown(description='From:', options=('degreeFahrenheit', 'degreeCelsius'), value='degreeFahrenheit')

Dropdown(description='To:', options=('degreeCelsius', 'degreeFahrenheit'), value='degreeCelsius')

Button(description='Convert', style=ButtonStyle())

Text(value='-17.7777777777778', description='Converted Temperature:')

Text(value='Negative seventeen ', description='Number Converted to Words:')

Text(value='Error: SOAP cannot convert fractional values. Temperature rounded down.', description='Error:')

Button(description='Open Web App in Separate Window', style=ButtonStyle())

In [20]:
!voila app.ipynb

[Voila] Using /tmp to store connection files
[Voila] Storing connection files in /tmp/voila_i4d5lzfz.
[Voila] Serving static files from /anaconda/envs/azureml_py38/lib/python3.8/site-packages/voila/static.
[Voila] Voilà is running at:
http://localhost:8866/
[Voila] WARNING | No web browser found: could not locate runnable browser.
^C
[Voila] Stopping...
